In [1]:
import enum
import glob
import os
from hashlib import new
from pathlib import Path
import time
from itertools import product

import functools

import numpy as np
import pandas as pd
import scipy
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from snorkel.labeling.model import LabelModel as LMsnorkel
from snorkel.labeling.model import MajorityLabelVoter

In [2]:
from sklearn.exceptions import UndefinedMetricWarning

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
def list2Nested(l, nested_length):
    return [l[i:i+nested_length] for i in range(0, len(l), nested_length)]

In [7]:
# Fetch UMLS ranks

sum_lf_p = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v3/summaries/lf_p_summary_tuipio3_train.csv'
sum_lf_i = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v3/summaries/lf_i_summary_tuipio3_train.csv'
sum_lf_o = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v3/summaries/lf_o_summary_tuipio3_train.csv'


def fetchRank(sum_lf_d, pattern):
    
    ranked_umls_coverage = dict()    
    umls_coverage_ = dict()
    
    data=pd.read_csv(sum_lf_d, sep='\t')
    
    for index, row in data.iterrows():
        if row[0].startswith(pattern):
            umls_coverage_[row[0]] = row[3]
    
    umls_coverage_sorted = sorted(umls_coverage_.items(), key=lambda x: x[1], reverse=True)
    
    for i in umls_coverage_sorted:
        k = str(i[0]).split('_')[-1]
        ranked_umls_coverage[k] = i[1]

    return ranked_umls_coverage


# fuzzy UMLS
ranksorted_p_umls_fuzzy = fetchRank(sum_lf_p, 'UMLS_fuzzy_')
ranksorted_i_umls_fuzzy = fetchRank(sum_lf_i, 'UMLS_fuzzy_')
ranksorted_o_umls_fuzzy = fetchRank(sum_lf_o, 'UMLS_fuzzy_')


# direct UMLS
ranksorted_p_umls_direct = fetchRank(sum_lf_p, 'UMLS_direct_')
ranksorted_i_umls_direct = fetchRank(sum_lf_i, 'UMLS_direct_')
ranksorted_o_umls_direct = fetchRank(sum_lf_o, 'UMLS_direct_')

In [8]:
# Partition LF's

def partitionLFs(umls_d):
    
    keys = list(umls_d.keys())

    partitioned_lfs = [ ]
    
    for i in range( 0, len(keys) ):

        if i == 0 or i == len(keys):
            if i == 0:
                partitioned_lfs.append( [keys] )
            if i ==len(keys):
                temp3 = list2Nested(keys, 1)
                partitioned_lfs.append( temp3 )
        else:
            temp1, temp2 = keys[:i] , keys[i:]
            temp3 = list2Nested( keys[:i], 1)
            temp3.append( keys[i:] )
            partitioned_lfs.append( temp3 )
    
    return partitioned_lfs


partitioned_p_umls_fuzzy = partitionLFs(ranksorted_p_umls_fuzzy)
partitioned_i_umls_fuzzy = partitionLFs(ranksorted_i_umls_fuzzy)
partitioned_o_umls_fuzzy = partitionLFs(ranksorted_o_umls_fuzzy)

partitioned_p_umls_direct = partitionLFs(ranksorted_p_umls_direct)
partitioned_i_umls_direct = partitionLFs(ranksorted_i_umls_direct)
partitioned_o_umls_direct = partitionLFs(ranksorted_o_umls_direct)

In [9]:
labelModel_mapper_LF = {1:1, -1:0, 0:-1}

In [12]:
import LMutils

train_file = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v3/gt/train_ebm_labels_tui_pio3.tsv'
training_data = pd.read_csv(train_file, sep='\t', header=0)

ebm_test_file = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v3/gt/test_ebm_labels_tui_pio3.tsv'
test_ebm_data = pd.read_csv(ebm_test_file, sep='\t', header=0)

physio_test_file = '/mnt/nas2/results/Results/systematicReview/distant_pico/EBM_PICO_GT/v3/gt/test_physio_labels_tui_pio3.tsv'
test_physio_data = pd.read_csv(physio_test_file, sep='\t', header=0)

In [13]:
from sklearn.model_selection import train_test_split
import itertools
import ast

train_df_tokens = [ word for index, value in training_data.tokens.items() for word in ast.literal_eval(value) ]
train_df_offsets = [ word for index, value in training_data.offsets.items() for word in ast.literal_eval(value) ]
train_df_p = [ int(lab) for index, value in training_data.p.items() for lab in ast.literal_eval(value) ]
train_df_i = [ int(lab) for index, value in training_data.i.items() for lab in ast.literal_eval(value) ]
train_df_o = [ int(lab) for index, value in training_data.o.items() for lab in ast.literal_eval(value) ]

In [14]:
data_df = pd.DataFrame({'tokens' : train_df_tokens,
                        'offsets': train_df_offsets,
                        'p' : train_df_p,
                        'i' : train_df_i,
                        'o' : train_df_o})

In [15]:
train_df, val_df = train_test_split(data_df, test_size=0.2, shuffle=False)

In [16]:
sents = [
    train_df.tokens.to_numpy() ,
    val_df.tokens.to_numpy() ,
    test_ebm_data.tokens.to_numpy() ,
    test_physio_data.tokens.to_numpy()    
]

offsets = [
    train_df.offsets.to_numpy() ,
    val_df.offsets.to_numpy() ,
    test_ebm_data.offsets.to_numpy() ,
    test_physio_data.offsets.to_numpy() 
]

Y_p = [
    train_df.p.to_numpy() ,
    val_df.p.to_numpy() ,
    test_ebm_data.p.to_numpy() ,
    test_physio_data.p.to_numpy() 
]


Y_i = [
    train_df.i.to_numpy() ,
    val_df.i.to_numpy() ,
    test_ebm_data.i.to_numpy() ,
    test_physio_data.i.to_numpy() 
]

Y_o = [
    train_df.o.to_numpy() ,
    val_df.o.to_numpy() ,
    test_ebm_data.o.to_numpy() ,
    test_physio_data.o.to_numpy() 
]

In [17]:
def df_to_list(data_column):
    return [ word for index, value in data_column.items() for word in ast.literal_eval(value) ]

In [18]:
def df_to_array(data_column):
    return np.array( [ word for index, value in data_column.items() for word in ast.literal_eval(value) ] )

In [19]:
def dict_to_array(label_column):
    return np.array( [ labelModel_mapper_LF[int(lab)] for index, value in label_column.items() for k, lab in ast.literal_eval(value).items() ] )

In [20]:
indir = '/mnt/nas2/results/Results/systematicReview/distant_pico/training_ebm_candidate_generation'
pathlist = Path(indir).glob('**/*.tsv')

tokens = ''

lfs = dict()
lfs_lm = dict()

for counter, file in enumerate(pathlist):
    
    k = str( file ).split('candidate_generation/v3/')[-1].replace('.tsv', '').replace('/', '_')
    mypath = Path(file)
    if mypath.stat().st_size != 0:
        data = pd.read_csv(file, sep='\t', header=0)
        
        data_tokens = data.tokens
        if len(tokens) < 5:
            tokens = df_to_array(data_tokens)
        
        data_labels = data.labels
        labels = dict_to_array(data_labels)
        lfs[k] = labels


print( 'Total number of tokens in validation set: ', len(tokens) )
print( 'Total number of LFs in the dictionary', len(lfs) )

Total number of tokens in validation set:  1303076
Total number of LFs in the dictionary 835


In [21]:
def lf_levels(umls_d:dict, pattern:str, picos:str):

    umls_level = dict()

    for key, value in umls_d.items():   # iter on both keys and values
        search_pattern = pattern + picos
        if key.startswith(search_pattern):
            k = str(key).split('_')[-1]
            umls_level[ k ] = value

    return umls_level


# Level 1: UMLS
umls_p = [
    lf_levels(lfs, name, 'P') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_i = [
    lf_levels(lfs, name, 'I') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]

umls_o = [
    lf_levels(lfs, name, 'O') 
    for i, name in enumerate(['UMLS_direct_', 'UMLS_fuzzy_'])
]


# Level 2: non UMLS
nonumls_p = [
    lf_levels(lfs, name, 'P') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_i = [
    lf_levels(lfs, name, 'I') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

nonumls_o = [
    lf_levels(lfs, name, 'O') 
    for i, name in enumerate(['nonUMLS_direct_', 'nonUMLS_fuzzy_'])
]

# Level 3: DS
ds_p = [
    lf_levels(lfs, name, 'P') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_i = [
    lf_levels(lfs, name, 'I') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]

ds_o = [
    lf_levels(lfs, name, 'O') 
    for i, name in enumerate(['ds_direct_', 'ds_fuzzy_'])
]


# Level 4: dictionary, rules, heuristics
heur_p = [
    lf_levels(lfs, name, 'P') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_i = [
    lf_levels(lfs, name, 'I') 
    for i, name in enumerate(['heuristics_direct_'])
]

heur_o = [
    lf_levels(lfs, name, 'O') 
    for i, name in enumerate(['heuristics_direct_'])
]


dict_p = [
    lf_levels(lfs, name, 'P') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_i = [
    lf_levels(lfs, name, 'I') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

dict_o = [
    lf_levels(lfs, name, 'O') 
    for i, name in enumerate(['dictionary_direct_', 'dictionary_fuzzy_'])
]

In [22]:
param_grid = {
    'lr': [0.001, 0.0001],
    'l2': [0.001, 0.0001],
    'n_epochs': [50, 100, 200, 600, 700, 1000, 2000],
    'prec_init': [0.6, 0.7, 0.8, 0.9],
    'optimizer': ["adamax", "adam", "sgd"],
    'lr_scheduler': ['constant'],
}

In [23]:
def sample_param_grid(param_grid, seed):
    """ Sample parameter grid
    :param param_grid:
    :param seed:
    :return:
    """
    rstate = np.random.get_state()
    np.random.seed(seed)
    params = list(product(*[param_grid[name] for name in param_grid]))
    np.random.shuffle(params)
    np.random.set_state(rstate)
    return params

In [24]:
def compare(s, t):
    return sorted(s) == sorted(t)

def getLFs(partition:list, umls_d:dict, seed_len:int):

    all_lfs_combined = []
    
    for lf in partition: # for each lf in a partition
        
        combine_here = [0] * seed_len

        for sab in lf:
            new_a = list(umls_d[sab])
            old_a = combine_here
            temp_a = []
            
            for o_a, n_a in zip(old_a, new_a):
                               
                if compare([o_a, n_a] ,[-1, 1]) == True:
                    replace_a = max( o_a, n_a )
                    temp_a.append( replace_a )
                    
                elif compare([o_a, n_a] ,[0, 1]) == True:
                    replace_a = max( o_a, n_a )
                    temp_a.append( replace_a )
                    
                elif compare([o_a, n_a] ,[-1, 0]) == True:
                    replace_a = min( o_a, n_a )
                    temp_a.append( replace_a )
                else:
                    temp_a.append( o_a )

            combine_here = temp_a

        all_lfs_combined.append( combine_here )

    return all_lfs_combined

In [25]:
def grid_search(model_class,
                model_class_init,
                param_grid,
                train=None,
                dev=None,
                other_train=None,
                n_model_search=5,
                val_metric='f1_macro',
                seed=1234,
                checkpoint_gt_mv=False,
                tag_fmt_ckpnt='IO'):
    
    
    """Simple grid search helper function
    Parameters
    ----------
    model_class
    model_class_init
    param_grid
    train
    dev
    n_model_search
    val_metric
    seed

    Returns
    -------
    """
    
    L_train, Y_train = train
    L_dev, Y_dev = dev
    
    # sample configs
    params = sample_param_grid(param_grid, seed)[:n_model_search]
    
    defaults = {'seed': seed}
    best_score, best_config = 0.0, None
    print(f"Grid search over {len(params)} configs")
    
    for i, config in enumerate(params):
        print(f'[{i}] Label Model')
        config = dict(zip(param_grid.keys(), config))
        config.update({param: value for param, value in defaults.items() if param not in config})

        model = model_class(**model_class_init)
        model.fit(L_train, Y_dev, **config)
        
        y_pred = model.predict(L_dev)
        
        if tag_fmt_ckpnt == 'IO':
            y_gold = np.array([0 if y == 0 else 1 for y in Y_dev])
        else:
            y_gold = Y_dev
            
            
        if -1 in y_pred:
            print("Label model predicted -1 (TODO: this happens inconsistently)")
            continue
            
        # use internal label model scorer to score the prediction
        metrics = model.score(L=L_dev,
                              Y=y_gold,
                              metrics=['accuracy', 'precision', 'recall', 'f1', 'f1_macro'],
                              tie_break_policy='random')
        
    
        # compare learned model against MV on same labeled dev set
        # skip if LM less than MV
        if checkpoint_gt_mv:
            mv_metrics = model.score(L=L_dev,
                                  Y=y_gold,
                                  metrics=['accuracy', 'precision', 'recall', 'f1', 'f1_macro'],
                                  tie_break_policy='random')

            if metrics[val_metric] < mv_metrics[val_metric]:
                continue
                
        if not best_score or metrics[val_metric] > best_score[val_metric]:
            print(config)
            best_score = metrics
            best_config = config
            
            # print training set score if we have labeled data
            if np.any(Y_train):
                y_pred = model.predict(L_train)

                if tag_fmt_ckpnt == 'IO':
                    y_gold = np.array([0 if y == 0 else 1 for y in Y_train])
                else:
                    y_gold = Y_train

                metrics = model.score(L=L_train,
                                      Y=y_gold,
                                      metrics=['accuracy', 'precision', 'recall', 'f1', 'f1_macro'],
                                      tie_break_policy='random')

                print('[TRAIN] {}'.format(' | '.join([f'{m}: {v * 100:2.2f}' for m, v in metrics.items()])))

            print('[DEV]   {}'.format(' | '.join([f'{m}: {v * 100:2.2f}' for m, v in best_score.items()])))
            print('-' * 88)
            
            
    # retrain best model
    print('BEST')
    print(best_config)
    model = model_class(**model_class_init)
    
    
    model.fit(L_train, Y_dev, **best_config)
    return model, best_config, best_score

In [26]:
def train(partitioned_d_umls, umls_d, non_umls_d, ds_d, heur_d, dict_d, Y_d, picos, paramgrid):
   

    best_f1_macro = 0.0
    best_overall_model = ''
    best_overall_config = ''
    
    
    model_class_init = {
        'cardinality': 2, 
        'verbose': True
    }

    num_hyperparams = functools.reduce(lambda x,y:x*y, [len(x) for x in param_grid.values()])
    print("Hyperparamater Search Space:", num_hyperparams)
    n_model_search = 50
    

    '''#########################################################################
    # Choosing the number of LF's from UMLS all
    #########################################################################'''
    
    for i, partition in enumerate(partitioned_d_umls):

        combined_lf = getLFs(partition, umls_d, len(train_df_tokens))
        assert len(partition) == len(combined_lf)
        print( 'Total number of UMLS partitions: ', len(partition) )
        
        combined_lf.extend( list(non_umls_d.values()) ) # Combine with level 2
        combined_lf.extend( list(ds_d.values()) ) # Combine with level 3
        combined_lf.extend( list(heur_d.values()) ) # Combine with level 4
        combined_lf.extend( list(dict_d.values()) ) # combine with level 4
                   
        L = np.array( combined_lf )
        L = np.transpose(L)
        L_train, L_val = train_test_split(L, test_size=0.20, shuffle=False)
        
        Y_train = Y_d[0]
        Y_val = Y_d[1]
        
        best_model, best_config, best_score = grid_search(LMsnorkel, 
                                               model_class_init, 
                                               paramgrid,
                                               train = (L_train, Y_train),
                                               dev = (L_val, Y_val),
                                               n_model_search=n_model_search, 
                                               val_metric='f1_macro', 
                                               seed=1234,
                                               tag_fmt_ckpnt='IO')

        if best_score['f1_macro'] > best_f1_macro:
            best_f1_macro = best_score['f1_macro']
            best_overall_model = best_model
            best_overall_config = best_config


        print('Best overall macro F1 score: ', best_f1_macro)
        print('Best overall configuration: ', best_overall_config)

In [161]:
train(partitioned_p_umls_fuzzy, umls_p[1], nonumls_p[1], ds_p[1], heur_p[0], dict_p[1], Y_p, 'p', paramgrid = param_grid)

Hyperparamater Search Space: 336
Total number of UMLS partitions:  1
(1042460,)
(260616,)
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 83.16 | precision: 32.02 | recall: 29.29 | f1: 30.60 | f1_macro: 60.51
[DEV]   accuracy: 83.19 | precision: 31.01 | recall: 28.54 | f1: 29.72 | f1_macro: 60.09
----------------------------------------------------------------------------------------
[1] Label Model
[2] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 83.41 | precision: 32.57 | recall: 28.87 | f1: 30.61 | f1_macro: 60.59
[DEV]   accuracy: 83.44 | precision: 31.57 | recall: 28.22 | f1: 29.80 | f1_macro: 60.20
----------------------------------------------------------------------------------------
[3] Label Model
[4] Label Model
[5] Label Model
[

[TRAIN] accuracy: 85.43 | precision: 38.80 | recall: 25.86 | f1: 31.03 | f1_macro: 61.44
[DEV]   accuracy: 85.48 | precision: 37.61 | recall: 25.10 | f1: 30.11 | f1_macro: 61.00
----------------------------------------------------------------------------------------
[3] Label Model
[4] Label Model
[5] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.6, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 87.21 | precision: 48.99 | recall: 21.27 | f1: 29.66 | f1_macro: 61.31
[DEV]   accuracy: 87.27 | precision: 47.49 | recall: 20.87 | f1: 29.00 | f1_macro: 61.00
----------------------------------------------------------------------------------------
[6] Label Model
[7] Label Model
[8] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 50, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 87.01 | precision: 47.43 | recall: 22.86 | f1: 30.85 | f1_macro: 61.84
[DEV]   accuracy: 87.03 | pre

[22] Label Model
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 86.45 | precision: 43.63 | recall: 23.73 | f1: 30.74 | f1_macro: 61.62
[DEV]   accuracy: 86.46 | precision: 42.09 | recall: 23.04 | f1: 29.78 | f1_macro: 61.14
----------------------------------------------------------------------------------------
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}


[TRAIN] accuracy: 83.45 | precision: 32.14 | recall: 27.49 | f1: 29.63 | f1_macro: 60.13
[DEV]   accuracy: 83.50 | precision: 31.35 | recall: 27.23 | f1: 29.14 | f1_macro: 59.90
----------------------------------------------------------------------------------------
[1] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.9, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 84.70 | precision: 35.96 | recall: 26.58 | f1: 30.57 | f1_macro: 60.98
[DEV]   accuracy: 84.71 | precision: 34.76 | recall: 25.89 | f1: 29.68 | f1_macro: 60.55
----------------------------------------------------------------------------------------
[2] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 85.81 | precision: 40.20 | recall: 24.55 | f1: 30.48 | f1_macro: 61.29
[DEV]   accuracy: 85.83 | precision: 38.76 | recall: 23.71 | f1: 29.42 | f1_macro: 60.77
------

[49] Label Model
BEST
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
Best overall macro F1 score:  0.6142796100146325
Best overall configuration:  {'lr': 0.001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.6, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
Total number of UMLS partitions:  11
(1042460,)
(260616,)
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 83.44 | precision: 31.84 | recall: 26.88 | f1: 29.15 | f1_macro: 59.89
[DEV]   accuracy: 83.47 | precision: 30.92 | recall: 26.47 | f1: 28.52 | f1_macro: 59.59
----------------------------------------------------------------------------------------
[1] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.9, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accurac

...

{'lr': 0.001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.9, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 84.15 | precision: 34.54 | recall: 27.95 | f1: 30.90 | f1_macro: 60.97
[DEV]   accuracy: 84.18 | precision: 33.46 | recall: 27.31 | f1: 30.07 | f1_macro: 60.57
----------------------------------------------------------------------------------------
[2] Label Model
[3] Label Model
[4] Label Model
[5] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.6, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 84.70 | precision: 36.23 | recall: 27.23 | f1: 31.09 | f1_macro: 61.24
[DEV]   accuracy: 84.68 | precision: 34.90 | recall: 26.56 | f1: 30.17 | f1_macro: 60.78
----------------------------------------------------------------------------------------
[6] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 200, 'prec_init': 0.6, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN]

[TRAIN] accuracy: 86.83 | precision: 45.95 | recall: 22.01 | f1: 29.76 | f1_macro: 61.25
[DEV]   accuracy: 86.85 | precision: 44.27 | recall: 21.44 | f1: 28.89 | f1_macro: 60.82
----------------------------------------------------------------------------------------
[7] Label Model
[8] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 50, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 85.01 | precision: 37.30 | recall: 26.81 | f1: 31.20 | f1_macro: 61.39
[DEV]   accuracy: 85.00 | precision: 35.98 | recall: 26.14 | f1: 30.28 | f1_macro: 60.94
----------------------------------------------------------------------------------------
[9] Label Model
[10] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 86.38 | precision: 43.34 | recall: 24.16 | f1: 31.02 | f1_macro: 61.73
[DEV]   accuracy: 86.39 | precision: 41.76 | recall: 23.48 | 

[12] Label Model
[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
Best overall macro F1 score:  0.6142796100146325
Best overall configuration:  {'lr': 0.001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.6, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
Total number of UML

Total number of UMLS partitions:  125
(1042460,)
(260616,)
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 83.62 | precision: 33.05 | recall: 28.55 | f1: 30.63 | f1_macro: 60.67
[DEV]   accuracy: 83.68 | precision: 32.16 | recall: 27.93 | f1: 29.89 | f1_macro: 60.33
----------------------------------------------------------------------------------------
[1] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.9, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 84.16 | precision: 34.55 | recall: 27.97 | f1: 30.92 | f1_macro: 60.98
[DEV]   accuracy: 84.18 | precision: 33.46 | recall: 27.31 | f1: 30.07 | f1_macro: 60.58
----------------------------------------------------------------------------------------
[2] Label Model
[3] Label Model
[4] Label Model
[5] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_e

{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.6, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 84.67 | precision: 36.13 | recall: 27.27 | f1: 31.08 | f1_macro: 61.23
[DEV]   accuracy: 84.66 | precision: 34.85 | recall: 26.61 | f1: 30.17 | f1_macro: 60.78
----------------------------------------------------------------------------------------
[6] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 200, 'prec_init': 0.6, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 86.83 | precision: 45.91 | recall: 22.03 | f1: 29.77 | f1_macro: 61.25
[DEV]   accuracy: 86.84 | precision: 44.19 | recall: 21.47 | f1: 28.90 | f1_macro: 60.82
----------------------------------------------------------------------------------------
[7] Label Model
[8] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 50, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 84.99 | precision: 37.

In [ ]:
train(partitioned_i_umls_fuzzy, umls_i[1], nonumls_i[1], ds_i[1], heur_i[0], dict_i[1], Y_i, 'i', paramgrid = param_grid)

Hyperparamater Search Space: 336
Total number of UMLS partitions:  1
(1042460,)
(260616,)
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 84.68 | precision: 26.86 | recall: 33.15 | f1: 29.67 | f1_macro: 60.54
[DEV]   accuracy: 84.89 | precision: 25.92 | recall: 33.40 | f1: 29.19 | f1_macro: 60.37
----------------------------------------------------------------------------------------
[1] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.9, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 86.66 | precision: 31.10 | recall: 30.25 | f1: 30.67 | f1_macro: 61.65
[DEV]   accuracy: 86.91 | precision: 29.91 | recall: 30.07 | f1: 29.99 | f1_macro: 61.38
----------------------------------------------------------------------------------------
[2] Label Model
[3] Label Model
[4] Label Model
[5] Label Model
[

[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
Best overall macro F1 score:  0.6174213629566849
Best overall configuration:  {'lr': 0.0001, 'l2': 0.001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
Total number of UMLS partitions:  4
(1042460,)
(260616,)
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 84.68 | precision: 26.81 | recall: 32.98 | f1: 2

[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
Best overall macro F1 score:  0.6188916272275283
Best overall configuration:  {'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
Total number of UMLS partitions:  7
(104

[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
Best overall macro F1 score:  0.6220023610425133
Best overall configuration:  {'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
Total number of UMLS partitions:  10
(10

[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
Best overall macro F1 score:  0.6229957875237966
Best overall configuration:  {'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
Total number of UMLS partitions:  13
(10

...

Best overall macro F1 score:  0.6229957875237966
Best overall configuration:  {'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
Total number of UMLS partitions:  89
(1042460,)
(260616,)
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 82.77 | precision: 25.50 | recall: 39.89 | f1: 31.11 | f1_macro: 60.63
[DEV]   accuracy: 82.92 | precision: 24.57 | recall: 40.21 | f1: 30.51 | f1_macro: 60.38
----------------------------------------------------------------------------------------
[1] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.9, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 86.45 | precision: 30.72 | recall: 31.07 | f1: 30.89 | f1_macro: 61.69
[DEV]   accuracy: 86.65 | precision: 29.49 | recall: 31.05 | f1: 30.25 | 

{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 86.28 | precision: 30.41 | recall: 31.57 | f1: 30.98 | f1_macro: 61.68
[DEV]   accuracy: 86.65 | precision: 29.85 | recall: 32.00 | f1: 30.89 | f1_macro: 61.75
----------------------------------------------------------------------------------------
[3] Label Model
[4] Label Model
[5] Label Model
[6] Label Model
[7] Label Model
[8] Label Model
[9] Label Model
[10] Label Model
[11] Label Model
[12] Label Model
[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39

[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
Best overall macro F1 score:  0.6229957875237966
Best overall configuration:  {'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
Total number of UMLS partitions:  96
(1042460,)
(260616,)
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 82.70 | precis

{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 82.67 | precision: 25.36 | recall: 39.98 | f1: 31.03 | f1_macro: 60.56
[DEV]   accuracy: 82.81 | precision: 24.44 | recall: 40.32 | f1: 30.43 | f1_macro: 60.32
----------------------------------------------------------------------------------------
[1] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.9, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 86.42 | precision: 30.81 | recall: 31.51 | f1: 31.16 | f1_macro: 61.81
[DEV]   accuracy: 86.61 | precision: 29.51 | recall: 31.39 | f1: 30.42 | f1_macro: 61.51
----------------------------------------------------------------------------------------
[2] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 86.30 | precision: 30.46 | recall: 31.54

[TRAIN] accuracy: 86.30 | precision: 30.46 | recall: 31.53 | f1: 30.98 | f1_macro: 61.69
[DEV]   accuracy: 86.67 | precision: 29.89 | recall: 31.94 | f1: 30.88 | f1_macro: 61.75
----------------------------------------------------------------------------------------
[3] Label Model
[4] Label Model
[5] Label Model
[6] Label Model
[7] Label Model
[8] Label Model
[9] Label Model
[10] Label Model
[11] Label Model
[12] Label Model
[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Labe

In [27]:
train(partitioned_o_umls_fuzzy, umls_o[1], nonumls_o[1], ds_o[1], heur_o[0], dict_o[1], Y_o, 'o', paramgrid = param_grid)

Hyperparamater Search Space: 336
Total number of UMLS partitions:  1
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 85.21 | precision: 26.32 | recall: 13.29 | f1: 17.66 | f1_macro: 54.77
[DEV]   accuracy: 85.25 | precision: 27.26 | recall: 13.67 | f1: 18.21 | f1_macro: 55.05
----------------------------------------------------------------------------------------
[1] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.9, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 80.46 | precision: 25.12 | recall: 32.14 | f1: 28.20 | f1_macro: 58.44
[DEV]   accuracy: 80.55 | precision: 25.74 | recall: 32.86 | f1: 28.87 | f1_macro: 58.80
----------------------------------------------------------------------------------------
[2] Label Model
[3] Label Model
[4] Label Model
[5] Label Model
{'lr': 0.0001, 'l2': 0

Best overall macro F1 score:  0.5910333167877516
Best overall configuration:  {'lr': 0.001, 'l2': 0.0001, 'n_epochs': 100, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
Total number of UMLS partitions:  4
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 79.51 | precision: 24.18 | recall: 33.56 | f1: 28.11 | f1_macro: 58.08
[DEV]   accuracy: 79.54 | precision: 24.65 | recall: 34.20 | f1: 28.65 | f1_macro: 58.35
----------------------------------------------------------------------------------------
[1] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.9, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 81.08 | precision: 25.65 | recall: 30.83 | f1: 28.00 | f1_macro: 58.56
[DEV]   accuracy: 81.07 | precision: 26.05 | recall: 31.32 | f1: 28.44 | f1_macro: 58.77
-----

[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.6, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
Best overall macro F1 score:  0.5963771603561658
Best overall configuration:  {'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.6, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
Total number of UMLS partitions:  7
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 77.48 | precision: 22.48 | recall: 36.21 | f1: 27.74 | f1_macro: 57.20
[DEV]   accuracy: 77.50 | precision: 22.91 | recall: 36.92 | f1: 28.27 | f1_macro: 57.47
----------------------------------------------------------------------------------------
[1] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.9, 'optimiz

[TRAIN] accuracy: 76.49 | precision: 21.68 | recall: 37.12 | f1: 27.38 | f1_macro: 56.68
[DEV]   accuracy: 76.51 | precision: 22.10 | recall: 37.87 | f1: 27.91 | f1_macro: 56.94
----------------------------------------------------------------------------------------
[1] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.9, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 80.10 | precision: 24.34 | recall: 31.64 | f1: 27.51 | f1_macro: 57.99
[DEV]   accuracy: 80.12 | precision: 24.81 | recall: 32.29 | f1: 28.06 | f1_macro: 58.26
----------------------------------------------------------------------------------------
[2] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 81.53 | precision: 25.68 | recall: 28.91 | f1: 27.20 | f1_macro: 58.31
[DEV]   accuracy: 81.51 | precision: 26.06 | recall: 29.36 | f1: 27.61 | f1_macro: 58.51
------

[7] Label Model
[8] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 50, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 82.54 | precision: 28.01 | recall: 29.49 | f1: 28.73 | f1_macro: 59.39
[DEV]   accuracy: 82.46 | precision: 28.32 | recall: 30.07 | f1: 29.17 | f1_macro: 59.58
----------------------------------------------------------------------------------------
[9] Label Model
[10] Label Model
[11] Label Model
[12] Label Model
[13] Label Model
Label model predicted -1 (TODO: this happens inconsistently)
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
Label model predicted -1 (TODO: this happ

...

{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 100, 'prec_init': 0.8, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 81.79 | precision: 25.32 | recall: 26.96 | f1: 26.11 | f1_macro: 57.86
[DEV]   accuracy: 81.78 | precision: 25.67 | recall: 27.27 | f1: 26.44 | f1_macro: 58.02
----------------------------------------------------------------------------------------
[3] Label Model
[4] Label Model
[5] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.6, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 79.22 | precision: 23.60 | recall: 33.13 | f1: 27.56 | f1_macro: 57.72
[DEV]   accuracy: 79.28 | precision: 24.13 | recall: 33.82 | f1: 28.17 | f1_macro: 58.03
----------------------------------------------------------------------------------------
[6] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 200, 'prec_init': 0.6, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 77.75

[4] Label Model
[5] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 700, 'prec_init': 0.6, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 79.21 | precision: 23.59 | recall: 33.14 | f1: 27.56 | f1_macro: 57.71
[DEV]   accuracy: 79.28 | precision: 24.12 | recall: 33.83 | f1: 28.16 | f1_macro: 58.03
----------------------------------------------------------------------------------------
[6] Label Model
{'lr': 0.0001, 'l2': 0.001, 'n_epochs': 200, 'prec_init': 0.6, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 77.75 | precision: 24.02 | recall: 39.95 | f1: 30.01 | f1_macro: 58.39
[DEV]   accuracy: 77.70 | precision: 24.28 | recall: 40.46 | f1: 30.35 | f1_macro: 58.54
----------------------------------------------------------------------------------------
[7] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 1000, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 83.80

[TRAIN] accuracy: 77.74 | precision: 24.02 | recall: 39.99 | f1: 30.01 | f1_macro: 58.39
[DEV]   accuracy: 77.68 | precision: 24.28 | recall: 40.51 | f1: 30.36 | f1_macro: 58.54
----------------------------------------------------------------------------------------
[7] Label Model
{'lr': 0.001, 'l2': 0.001, 'n_epochs': 1000, 'prec_init': 0.9, 'optimizer': 'sgd', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 83.80 | precision: 28.72 | recall: 24.09 | f1: 26.21 | f1_macro: 58.55
[DEV]   accuracy: 83.78 | precision: 29.11 | recall: 24.41 | f1: 26.55 | f1_macro: 58.72
----------------------------------------------------------------------------------------
[8] Label Model
[9] Label Model
[10] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 81.38 | precision: 26.22 | recall: 30.91 | f1: 28.37 | f1_macro: 58.83
[DEV]   accuracy: 81.33 | precision: 26.61 | recall: 31.57 |

[12] Label Model
[13] Label Model
[14] Label Model
[15] Label Model
[16] Label Model
[17] Label Model
[18] Label Model
[19] Label Model
[20] Label Model
[21] Label Model
[22] Label Model
[23] Label Model
[24] Label Model
[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
Best overall macro F1 score:  0.5964504382266139
Best overall configuration:  {'lr': 0.0001, 'l2': 0.001, 'n_epochs': 600, 'prec_init': 0.6, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
Total number of UM

[25] Label Model
[26] Label Model
[27] Label Model
[28] Label Model
[29] Label Model
[30] Label Model
[31] Label Model
[32] Label Model
[33] Label Model
[34] Label Model
[35] Label Model
[36] Label Model
[37] Label Model
[38] Label Model
[39] Label Model
[40] Label Model
[41] Label Model
[42] Label Model
[43] Label Model
[44] Label Model
[45] Label Model
[46] Label Model
[47] Label Model
[48] Label Model
[49] Label Model
BEST
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 50, 'prec_init': 0.8, 'optimizer': 'adamax', 'lr_scheduler': 'constant', 'seed': 1234}
Best overall macro F1 score:  0.5964504382266139
Best overall configuration:  {'lr': 0.0001, 'l2': 0.001, 'n_epochs': 600, 'prec_init': 0.6, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
Total number of UMLS partitions:  126
Grid search over 50 configs
[0] Label Model
{'lr': 0.001, 'l2': 0.0001, 'n_epochs': 200, 'prec_init': 0.8, 'optimizer': 'adam', 'lr_scheduler': 'constant', 'seed': 1234}
[TRAIN] accuracy: 70.78 | preci